In [1]:
import numpy as np
import pandas as pd
import scipy.io
from scipy import sparse
from sklearn.decomposition import PCA
from sklearn.linear_model import Ridge
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from scipy.stats import zscore
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import math
from matplotlib.pyplot import *

In [2]:
class Reservoir(object):
    """
    Build a reservoir and evaluate internal states
    
    Parameters:
        n_internal_units = processing units in the reservoir
        spectral_radius = largest eigenvalue of the reservoir matrix of connection weights
        leak = amount of leakage in the reservoir state update (optional)
        connectivity = percentage of nonzero connection weights (unused in circle reservoir)
        input_scaling = scaling of the input connection weights
        noise_level = deviation of the Gaussian noise injected in the state update
        circle = generate determinisitc reservoir with circle topology
    """
    
    def __init__(self, n_internal_units=100, spectral_radius=0.99, leak=None,
                 connectivity=0.3, input_scaling=0.2, noise_level=0.01, circle=False):
        
        # Initialize attributes
        self._n_internal_units = n_internal_units
        self._input_scaling = input_scaling
        self._noise_level = noise_level
        self._leak = leak

        # Input weights depend on input size: they are set when data is provided
        self._input_weights = None

        # Generate internal weights
        if circle:
            self._internal_weights = self._initialize_internal_weights_Circ(
                    n_internal_units,
                    spectral_radius)
        else:
            self._internal_weights = self._initialize_internal_weights(
                n_internal_units,
                connectivity,
                spectral_radius)


    def _initialize_internal_weights_Circ(self, n_internal_units, spectral_radius):
        
        internal_weights = np.zeros((n_internal_units, n_internal_units))
        internal_weights[0,-1] = spectral_radius
        for i in range(n_internal_units-1):
            internal_weights[i+1,i] = spectral_radius
                
        return internal_weights
    
    
    def _initialize_internal_weights(self, n_internal_units,
                                     connectivity, spectral_radius):

        # Generate sparse, uniformly distributed weights.
        internal_weights = sparse.rand(n_internal_units,
                                       n_internal_units,
                                       density=connectivity).todense()

        # Ensure that the nonzero values are uniformly distributed in [-0.5, 0.5]
        internal_weights[np.where(internal_weights > 0)] -= 0.5
        
        # Adjust the spectral radius.
        E, _ = np.linalg.eig(internal_weights)
        e_max = np.max(np.abs(E))
        internal_weights /= np.abs(e_max)/spectral_radius       

        return internal_weights


    def _compute_state_matrix(self, X, n_drop=0):
        N, T, _ = X.shape
        previous_state = np.zeros((N, self._n_internal_units), dtype=float)

        # Storage
        state_matrix = np.empty((N, T - n_drop, self._n_internal_units), dtype=float)
        for t in range(T):
            current_input = X[:, t, :]

            # Calculate state
            state_before_tanh = self._internal_weights.dot(previous_state.T) + self._input_weights.dot(current_input.T)

            # Add noise
            state_before_tanh += np.random.rand(self._n_internal_units, N)*self._noise_level

            # Apply nonlinearity and leakage (optional)
            if self._leak is None:
                previous_state = np.tanh(state_before_tanh).T
            else:
                previous_state = (1.0 - self._leak)*previous_state + np.tanh(state_before_tanh).T

            # Store everything after the dropout period
            if (t > n_drop - 1):
                state_matrix[:, t - n_drop, :] = previous_state

        return state_matrix


    def get_states(self, X, n_drop=0, bidir=True):
        N, T, V = X.shape
        if self._input_weights is None:
            self._input_weights = (2.0*np.random.binomial(1, 0.5 , [self._n_internal_units, V]) - 1.0)*self._input_scaling

        # compute sequence of reservoir states
        states = self._compute_state_matrix(X, n_drop)
    
        # reservoir states on time reversed input
        if bidir is True:
            X_r = X[:, ::-1, :]
            states_r = self._compute_state_matrix(X_r, n_drop)
            states = np.concatenate((states, states_r), axis=2)

        return states
    
    def getReservoirEmbedding(self, X,pca, ridge_embedding,  n_drop=5, bidir=True, test = False):

        res_states = self.get_states(X, n_drop=5, bidir=True)


        N_samples = res_states.shape[0]
        res_states = res_states.reshape(-1, res_states.shape[2])                   
        # ..transform..
        if test:
            red_states = pca.transform(res_states)
        else:
            red_states = pca.fit_transform(res_states)          
        # ..and put back in tensor form
        red_states = red_states.reshape(N_samples,-1,red_states.shape[1])  

        coeff_tr = []
        biases_tr = []   

        for i in range(X.shape[0]):
            ridge_embedding.fit(red_states[i, 0:-1, :], red_states[i, 1:, :])
            coeff_tr.append(ridge_embedding.coef_.ravel())
            biases_tr.append(ridge_embedding.intercept_.ravel())
        #print(np.array(coeff_tr).shape,np.array(biases_tr).shape)
        input_repr = np.concatenate((np.vstack(coeff_tr), np.vstack(biases_tr)), axis=1)
        return input_repr

In [3]:
def targetify(s):
    if s == 'Benign':
        return 0
    else:
        return 1

In [4]:
def eqArray(a,b):
    return np.where(a == b, 1, 0)

In [5]:
datasets = ["Thursday-15-02-2018_TrafficForML_CICFlowMeter.csv", "Friday-16-02-2018_TrafficForML_CICFlowMeter.csv"]

features_Th15022018 = ['Fwd Seg Size Min', 'Init Fwd Win Byts', 'Bwd Pkt Len Max', 'Fwd IAT Min', 'Bwd IAT Mean', 'Fwd IAT Tot', 'Flow IAT Mean', 'Flow Duration', 'Fwd IAT Mean', 'Pkt Len Max', 
                       'Fwd IAT Max', 'Bwd Pkt Len Std', 'PSH Flag Cnt', 'Flow IAT Min', 'Bwd IAT Max', 'ACK Flag Cnt', 'Flow IAT Max', 'Idle Max', 'Fwd Header Len', 'Flow IAT Std']
features_Fr16022018 = ['Fwd Pkt Len Std', 'Pkt Len Std', 'Fwd Seg Size Avg', 'Fwd Pkt Len Mean', 'Fwd Pkt Len Max', 'TotLen Fwd Pkts', 'Pkt Len Var', 'Bwd Pkt Len Max', 'Pkt Len Max', 'Pkt Size Avg',
                       'Bwd Seg Size Avg', 'Flow IAT Std', 'Bwd Pkt Len Mean', 'Bwd Pkt Len Std', 'Fwd IAT Max', 'PSH Flag Cnt', 'ACK Flag Cnt', 'Fwd IAT Std', 'Subflow Fwd Byts', 'Flow IAT Mean']

#numFeatures = [10, 15, 20]
#fracOfData = [0.5, 0.75, 1]
#numInternalUnits = [5, 10, 15, 20]

In [6]:
dataset = "Friday-16-02-2018_TrafficForML_CICFlowMeter.csv"
path = "../Datasets/Raw_Dataset/" + dataset
df = pd.read_csv(path)
num_features = 10
features = features_Fr16022018[0:num_features]
fraction = 0.75
n=5 #number of internal units

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (0,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
print(str(num_features) + " features")
print("fraction:" + str(fraction))
data = df.sample(frac=fraction, replace=True, random_state=1)

# get X and y. Normalize X and make it into 3D shape for reservoir
num_col = data.shape[1]
num_row = data.shape[0]

X_data = data[features]
X_data[features] = X_data[features].apply(pd.to_numeric, errors='coerce', axis=1)
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(X_data.values)
X = np.nan_to_num(x_scaled)
if len(X.shape) < 3:
    X = np.atleast_3d(X)
y = data['Label'].apply(targetify)
print("Finished loading X and y......")

# split into training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

print("X_train shape:" + str(X_train.shape), "y_train shape:" + str(y_train.shape))
print("X_test shape:" + str(X_test.shape), "y_test shape:" + str(y_test.shape))

pca = PCA() #n_components gives number of components to keep for linear dimensionality reduction
ridge_embedding = Ridge(alpha=10, fit_intercept=True)
readout = Ridge(alpha=5)

10 features
fraction:0.75


/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


Finished loading X and y......
X_train shape:(589823, 10, 1) y_train shape:(589823,)
X_test shape:(196608, 10, 1) y_test shape:(196608,)


In [8]:
print(str(n) + " internal units")
for input_scaling in [0.3]:
    for spectral_radius in [0.9]:
        for leaking_rate in [0.2, 0.6, 0.9]:
            print("input_scaling: " + str(input_scaling))
            print("spectral_radius: " + str(spectral_radius))
            print("leaking rate: " + str(leaking_rate))
            
            #run through reservoir
            res = Reservoir(n_internal_units=n, spectral_radius=spectral_radius, leak=leaking_rate,
                 connectivity=0.25, input_scaling=input_scaling, noise_level=0.01, circle=False)
            input_repr = res.getReservoirEmbedding(np.array(X_train), pca, ridge_embedding,  n_drop=5, bidir=False, test = False)
            print("Finished loading training reservoir embedding......")
            input_repr_te = res.getReservoirEmbedding(np.array(X_test), pca, ridge_embedding,  n_drop=5, bidir=False, test = True)
            print("Finished loading testing reservoir embedding......")

            #fit output
            readout.fit(input_repr, y_train)
            pred_class = readout.predict(input_repr_te)
            predictions = [int(round(x)) for x in pred_class]
            true_class = list(y_test)

            #analysis
            compdf = pd.DataFrame({'pred_class':pred_class, 'true_class':true_class})
            compdf = compdf.sort_values('pred_class', ascending=False)
            print(str(compdf.head(10)))
            compdf.to_csv(str(dataset.split('_')[0]) + 'parameters_' + str(input_scaling) + '_' + str(spectral_radius) + '_' + str(leaking_rate) + '.csv')
            accuracy = np.sum(list(map(eqArray, predictions, true_class))) / len(true_class)
            #f1 = f1_score(true_class, predictions)
            #auc = roc_auc_score(true_class, predictions)

            print("# of nonzero:" + str(np.count_nonzero(predictions)))
            print("accuracy is " + str(accuracy))
            #print("f1 is " + str(f1))
            #print("auc is " + str(auc))
            print("*******************************************************************")

5 internal units
input_scaling: 0.3
spectral_radius: 0.9
leaking rate: 0.2
Finished loading training reservoir embedding......
Finished loading testing reservoir embedding......
        pred_class  true_class
180463    1.223153           1
17673     1.198138           1
149165    1.196657           1
142340    1.195524           1
126047    1.194874           1
163117    1.193774           1
26547     1.193599           1
147536    1.192770           1
156163    1.188533           1
152437    1.185966           1
# of nonzero:114099
accuracy is 0.9958394368489584
*******************************************************************
input_scaling: 0.3
spectral_radius: 0.9
leaking rate: 0.6
Finished loading training reservoir embedding......
Finished loading testing reservoir embedding......
        pred_class  true_class
179655    2.119526           0
61894     1.116945           1
137979    1.105840           1
147647    1.102921           1
195552    1.099403           1
87694     1.094

# Predicting Hulk Attack

In [15]:
hulkdf = pd.read_csv("../CustomDatasets/RegularHulkAttack.csv")
hulkdf.head()

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,192.168.0.12-23.59.158.215-63949-443-6,23.59.158.215,443,192.168.0.12,63949,6,6/12/2019 9:20,1,1,1,...,0,0.000000e+00,0.000000e+00,0.0,0,0.000000e+00,0.000000e+00,0.0,0.0,Benign
1,192.168.0.12-192.168.0.15-0-0-0,192.168.0.15,0,192.168.0.12,0,0,6/12/2019 9:19,119471947,119,1,...,0,0.000000e+00,0.000000e+00,0.0,0,0.000000e+00,0.000000e+00,0.0,0.0,ModifiedHuLK
2,8.0.6.4-8.6.0.1-0-0-0,8.6.0.1,0,8.0.6.4,0,0,6/12/2019 9:19,115023918,23,1,...,0,4.249888e+06,6.028923e+06,15500000.0,126,6.886507e+06,3.949426e+06,19000000.0,5000053.0,Benign
3,192.168.0.1-239.255.255.250-51772-1900-17,192.168.0.1,51772,239.255.255.250,1900,17,6/12/2019 9:20,106023344,351,1,...,0,2.730851e+05,4.610152e+03,278251.0,266549,1.480000e+07,4.878160e+05,15700000.0,14100000.0,Benign
4,239.255.255.250-20.20.20.1-1900-33157-17,20.20.20.1,33157,239.255.255.250,1900,17,6/12/2019 9:20,106014555,351,1,...,0,2.647474e+05,4.230452e+03,270305.0,258567,1.480000e+07,4.878088e+05,15800000.0,14100000.0,Benign


In [16]:
features = ['Flow Duration', 'Tot Fwd Pkts',
       'Tot Bwd Pkts', 'TotLen Fwd Pkts', 'TotLen Bwd Pkts', 'Fwd Pkt Len Max',
       'Fwd Pkt Len Min', 'Fwd Pkt Len Mean', 'Fwd Pkt Len Std',
       'Bwd Pkt Len Max', 'Bwd Pkt Len Min', 'Bwd Pkt Len Mean',
       'Bwd Pkt Len Std', 'Flow Byts/s', 'Flow Pkts/s', 'Flow IAT Mean',
       'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Tot',
       'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min',
       'Bwd IAT Tot', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max',
       'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags',
       'Bwd URG Flags', 'Fwd Header Len', 'Bwd Header Len', 'Fwd Pkts/s',
       'Bwd Pkts/s', 'Pkt Len Min', 'Pkt Len Max', 'Pkt Len Mean',
       'Pkt Len Std', 'Pkt Len Var', 'FIN Flag Cnt', 'SYN Flag Cnt',
       'RST Flag Cnt', 'PSH Flag Cnt', 'ACK Flag Cnt', 'URG Flag Cnt',
       'CWE Flag Count', 'ECE Flag Cnt', 'Down/Up Ratio', 'Pkt Size Avg',
       'Fwd Seg Size Avg', 'Bwd Seg Size Avg', 'Fwd Byts/b Avg',
       'Fwd Pkts/b Avg', 'Fwd Blk Rate Avg', 'Bwd Byts/b Avg',
       'Bwd Pkts/b Avg', 'Bwd Blk Rate Avg', 'Subflow Fwd Pkts',
       'Subflow Fwd Byts', 'Subflow Bwd Pkts', 'Subflow Bwd Byts',
       'Init Fwd Win Byts', 'Init Bwd Win Byts', 'Fwd Act Data Pkts',
       'Fwd Seg Size Min', 'Active Mean', 'Active Std', 'Active Max',
       'Active Min', 'Idle Mean', 'Idle Std', 'Idle Max', 'Idle Min']

In [17]:
X_hulk_data = hulkdf[features]
X_hulk_data[features] = X_hulk_data[features].apply(pd.to_numeric, errors='coerce', axis=1)
min_max_scaler = preprocessing.MinMaxScaler()
x_hulk_scaled = min_max_scaler.fit_transform(X_hulk_data.values)
X_hulk = np.nan_to_num(x_hulk_scaled)
if len(X_hulk.shape) < 3:
    X_hulk = np.atleast_3d(X_hulk)
y_hulk = hulkdf['Label'].apply(targetify)

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [18]:
input_repr_te_hulk = res.getReservoirEmbedding(np.array(X_hulk), pca, ridge_embedding,  n_drop=5, bidir=False, test = True)
print("Finished loading testing reservoir embedding......")

Finished loading testing reservoir embedding......


In [19]:
pred_class_hulk = readout.predict(input_repr_te_hulk)
predictions_hulk = [int(round(x)) for x in pred_class_hulk]
true_class_hulk = list(y_hulk)

In [20]:
compdf_hulk = pd.DataFrame({'pred_class':pred_class_hulk, 'true_class':true_class_hulk})
compdf_hulk = compdf_hulk.sort_values('pred_class', ascending=False)

In [21]:
def myRound(x, r):
    if x>r/float(1000):
        return 1
    else:
        return 0

In [22]:
def eqArray(a,b):
    return np.where(a == b, 1, 0)

In [23]:
predictions_hulk = list(compdf_hulk['pred_class'].apply(myRound, r=225))
true_class_hulk = list(compdf_hulk['true_class'])
accuracy_hulk = np.sum(list(map(eqArray, predictions_hulk, true_class_hulk))) / len(true_class_hulk)
accuracy_hulk

0.9723242994947175

# Calculating Confusion Matrix

In [24]:
from sklearn.metrics import confusion_matrix

In [25]:
confm = confusion_matrix(true_class_hulk, predictions_hulk)
confm

array([[  17,  241],
       [   0, 8450]])

In [27]:
tn, fp, fn, tp = confm.ravel()
(tn + tp)/(tn+tp+fn+fp)

0.9723242994947175